# Storey Metrics

This notebook reads in netcdf files from previous notebook and adds metrics using the user defined storey limits for each plot.

In [1]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import rioxarray

data_dir = Path('../data')

In [2]:
# Read in user defined storey limits
cover_limits = pd.read_csv(data_dir / "outputs/lidar_assessed_storey_limits.csv")
cover_limits = cover_limits[['site_plot_id', 'la_ground_limit', 'la_understorey_limit', 'la_midstorey_limit']]
# Drop missing rows
cover_limits = cover_limits.dropna(subset=['la_ground_limit'])
cover_limits = cover_limits.set_index('site_plot_id')
cover_limits

,la_ground_limit,la_understorey_limit,la_midstorey_limit
site_plot_id,,,
AGG_O_01_P1,1.0,4.0,NaN
AGG_O_01_P2,1.0,4.0,NaN
AGG_O_01_P3,1.0,4.0,NaN
AGG_O_01_P4,1.0,10.0,30.0
AGG_O_01_P5,1.0,10.0,30.0
...,...,...,...
ULY_Y_96_P1,1.0,3.0,17.0
ULY_Y_96_P2,1.0,3.0,17.0
ULY_Y_96_P3,1.0,3.0,17.0


In [3]:
def create_storey_bounds(row: pd.Series):

    ground = (0, 1)

    lower_limit = 1

    understorey_limit = row['la_understorey_limit']
    if not pd.isna(understorey_limit):
        understorey_limit = int(understorey_limit)
        understorey = (lower_limit, understorey_limit)
        lower_limit = understorey_limit
    else:
        understorey = np.nan

    midstorey_limit = row['la_midstorey_limit']
    if not pd.isna(midstorey_limit):
        midstorey_limit = int(midstorey_limit)
        midstorey = (lower_limit, int(midstorey_limit))
        lower_limit = midstorey_limit
    else:
        midstorey = np.nan

    # Max of upperstorey dependent on plot
    upperstorey = (lower_limit, np.nan)


    return pd.Series({
        "ground": ground,
        "understorey": understorey,
        "midstorey": midstorey,
        "upperstorey": upperstorey
    })

cover_bounds = cover_limits.apply(create_storey_bounds, axis=1)
cover_bounds

,ground,understorey,midstorey,upperstorey
site_plot_id,,,,
AGG_O_01_P1,"(0, 1)","(1, 4)",NaN,"(4, nan)"
AGG_O_01_P2,"(0, 1)","(1, 4)",NaN,"(4, nan)"
AGG_O_01_P3,"(0, 1)","(1, 4)",NaN,"(4, nan)"
AGG_O_01_P4,"(0, 1)","(1, 10)","(10, 30)","(30, nan)"
AGG_O_01_P5,"(0, 1)","(1, 10)","(10, 30)","(30, nan)"
...,...,...,...,...
ULY_Y_96_P1,"(0, 1)","(1, 3)","(3, 17)","(17, nan)"
ULY_Y_96_P2,"(0, 1)","(1, 3)","(3, 17)","(17, nan)"
ULY_Y_96_P3,"(0, 1)","(1, 3)","(3, 17)","(17, nan)"


In [4]:
metrics_dir = data_dir / 'outputs/plots/metrics/x1-y1-z1/net_cdf'

def read_plot_metrics(plot_id: str):
    metrics = xr.open_dataset(metrics_dir / f"{plot_id}.nc", decode_coords='all')
    metrics.load()
    metrics.close()
    return metrics

In [5]:
def add_cover_metrics(metrics: xr.Dataset, storey: str, bounds: tuple[int, int]):
    lower, upper = bounds

    max_z = metrics['z'].max().item()

    if max_z < lower + 1:
        return metrics

    metrics_slice = metrics.sel(z=slice(lower + 1, upper))

    rel_density = metrics_slice['vox_rel_density'].sum(dim='z', min_count=1)
    rel_density_w = metrics_slice['vox_rel_density_w'].sum(dim='z', min_count=1)

    capture = metrics_slice['vox_inside'].sum(dim='z', min_count=1) / metrics_slice['vox_enter'].isel(z=-1)
    capture_w = metrics_slice['vox_inside_w'].sum(dim='z', min_count=1) / metrics_slice['vox_enter_w'].isel(z=-1)

    metrics[f'{storey}_rel_density'] = rel_density
    metrics[f'{storey}_rel_density_w'] = rel_density_w
    metrics[f'{storey}_capture'] = capture
    metrics[f'{storey}_capture_w'] = capture_w
    metrics.attrs[f'{storey}_bounds'] = str(bounds)

    return metrics

def add_storey_cover_metrics(metrics: xr.Dataset, cover_bounds: pd.Series):
    metrics = add_cover_metrics(metrics, 'groundstorey', cover_bounds['ground'])
    if not pd.isna(cover_bounds['understorey']):
        metrics = add_cover_metrics(metrics, 'understorey', cover_bounds['understorey'])
    if not pd.isna(cover_bounds['midstorey']):
        metrics = add_cover_metrics(metrics, 'midstorey', cover_bounds['midstorey'])
    metrics = add_cover_metrics(metrics, 'upperstorey', cover_bounds['upperstorey'])
    return metrics


metrics = read_plot_metrics('AGG_O_01_P1')
metrics = add_storey_cover_metrics(metrics, cover_bounds.loc['AGG_O_01_P1'])
metrics

<xarray.Dataset> Size: 8MB
Dimensions:                     (y: 65, x: 67, z: 18)
Coordinates:
  * x                           (x) float64 536B 4.63e+05 4.63e+05 ... 4.63e+05
  * y                           (y) float64 520B 5.26e+06 5.26e+06 ... 5.26e+06
    spatial_ref                 int64 8B 0
  * z                           (z) float64 144B 0.0 1.0 2.0 ... 15.0 16.0 17.0
Data variables: (12/53)
    point_density               (y, x) float64 35kB nan nan nan ... nan nan nan
    pulse_density               (y, x) float64 35kB nan nan nan ... nan nan nan
    scan_angle_mean             (y, x) float32 17kB nan nan nan ... nan nan nan
    chm                         (y, x) float64 35kB nan nan nan ... nan nan nan
    veg_height_mean             (y, x) float64 35kB nan nan nan ... nan nan nan
    veg_height_median           (y, x) float64 35kB nan nan nan ... nan nan nan
    ...                          ...
    understorey_capture         (y, x) float64 35kB nan nan nan ... nan nan nan
    understorey_capture_w       (y, x) float64 35kB nan nan nan ... nan nan nan
    upperstorey_rel_density     (y, x) float64 35kB nan nan nan ... nan nan nan
    upperstorey_rel_density_w   (y, x) float64 35kB nan nan nan ... nan nan nan
    upperstorey_capture         (y, x) float64 35kB nan nan nan ... nan nan nan
    upperstorey_capture_w       (y, x) float64 35kB nan nan nan ... nan nan nan
Attributes:
    xy_bin_size:          1
    z_bin_size:           1
    plot_id:              AGG_O_01_P1
    groundstorey_bounds:  (0, 1)
    understorey_bounds:   (1, 4)
    upperstorey_bounds:   (4, nan)

In [6]:
for site_plot_id, bounds in cover_bounds.iterrows():
    metrics = read_plot_metrics(site_plot_id)
    metrics_with_cover = add_storey_cover_metrics(metrics, bounds)
    metrics_with_cover.to_netcdf(metrics_dir / f"{site_plot_id}_with_cover.nc")
    metrics_with_cover.close()